In [ ]:
# 깃허브에서 데이터셋 다운로드하기
!git clone https://github.com/ndb796/Scene-Classification-Dataset-Split 
# 폴더 안으로 이동
%cd Scene-Classification-Dataset-Split

Cloning into 'Scene-Classification-Dataset-Split'...
remote: Enumerating objects: 17038, done.
remote: Total 17038 (delta 0), reused 0 (delta 0), pack-reused 17038
Receiving objects: 100% (17038/17038), 136.02 MiB | 39.39 MiB/s, done.
Checking out files: 100% (17035/17035), done.
/content/Scene-Classification-Dataset-Split


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('train/', train_transform)
test_dataset = datasets.ImageFolder('val/', test_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
def mixup(image, label):
  t = np.random.beta(0.2, 0.2)
  image[0] = t * image[0] + (1 - t) * image[-1]
  label[0] = t * label[0] + (1 - t) * label[-1]
  plt.imshow(  tensor_image.permute(1, 2, 0)  )

In [ ]:
def run(model, criterion, optimizer, data_loader, train=True):
  if train:
    model.train()
  else:
    model.eval()
  
  correct = 0
  total = 0
  loss_sum = 0

  for image, label in data_loader:
    image = image.to(device)
    label = label.to(device)

    output = model(image)
    loss = criterion(output, label)
    if train:
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    else:
      _, predict = output.max(1)
      correct += predict.eq(label).sum().item()
      total += label.size(0)
      loss_sum += loss.item()
  
  if not train:
    return correct / total, loss_sum / total

In [ ]:
learning_rate = 0.001
num_epoch = 50
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 6)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.0002)

train_data = {'accuracy':[], 'loss':[]}
test_data = {'accuracy':[], 'loss':[]}
for epoch in range(1, num_epoch + 1):
  print(f'Epoch {epoch}')
  
  try:
    run(model, criterion, optimizer, train_dataloader)
  except:
    break

  accuracy, loss = run(model, criterion, optimizer, train_dataloader, train=False)
  train_data['accuracy'].append(accuracy)
  train_data['loss'].append(loss)
  print(f'  Train : Accuracy {accuracy}, Loss {loss}')

  accuracy, loss = run(model, criterion, optimizer, test_dataloader, train=False)
  test_data['accuracy'].append(accuracy)
  test_data['loss'].append(loss)
  print(f'  Test : Accuracy {accuracy}, Loss {loss}')

Epoch 1
  Train : Accuracy 0.7451383283187789, Loss 0.06538488635787998
  Test : Accuracy 0.8024655121808042, Loss 0.0623529949319793
Epoch 2
  Train : Accuracy 0.7854993762383503, Loss 0.04256149930501532
  Test : Accuracy 0.8538303492808923, Loss 0.034854868534621095
Epoch 3
  Train : Accuracy 0.8105966096719748, Loss 0.03467523777306434
  Test : Accuracy 0.8664514235397711, Loss 0.02675003663147164
Epoch 4
  Train : Accuracy 0.8318778894841125, Loss 0.030744119368724832
  Test : Accuracy 0.8796595245083652, Loss 0.02321788190150842
Epoch 5
  Train : Accuracy 0.8400968665150069, Loss 0.0287808758679127
  Test : Accuracy 0.885529791605518, Loss 0.02123230333582971
Epoch 6
  Train : Accuracy 0.8483158435459015, Loss 0.02738730642823469
  Test : Accuracy 0.886410331670091, Loss 0.0206777975783255
Epoch 7
  Train : Accuracy 0.8549203786600132, Loss 0.02606873968214949
  Test : Accuracy 0.8958027590255356, Loss 0.019537507174501048
Epoch 8
  Train : Accuracy 0.8585161811110296, Loss 0.025

In [ ]:
torch.save(model, 'model/model.pt')

In [ ]:
%ls

model/  README.md  train/  val/


In [ ]:
%cd model

/content/Scene-Classification-Dataset-Split/model


In [ ]:
%cd ..

/content/Scene-Classification-Dataset-Split
